In [1]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Dataset Prep

In [2]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('/home/kosmas/projects/llm-in-cybersecurity/final-project/datasets/train_emails.csv')
val_df = pd.read_csv('/home/kosmas/projects/llm-in-cybersecurity/final-project/datasets/val_emails.csv')
test_df = pd.read_csv('/home/kosmas/projects/llm-in-cybersecurity/final-project/datasets/test_emails.csv')

# Display basic information
print(f"Training set: {train_df.shape[0]} examples")
print(f"Validation set: {val_df.shape[0]} examples")
print(f"Test set: {test_df.shape[0]} examples")

# Check for class balance
print("\nClass distribution:")
print("Training set:")
print(train_df['label'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}"))
print("\nValidation set:")
print(val_df['label'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}"))
print("\nTest set:")
print(test_df['label'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}"))

# Display a few examples with both ham and spam
def display_examples(df, label_value, n=3):
    examples = df[df['label'] == label_value].sample(n, random_state=42)
    return examples[['subject', 'body', 'label']]

# Display ham examples (label=0)
print("\nSample HAM emails:")
display(display_examples(train_df, 0))

# Display spam examples (label=1)
print("\nSample SPAM emails:")
display(display_examples(train_df, 1))

# Check for missing values
print("\nMissing values in train set:")
print(train_df.isnull().sum())
print("\nMissing values in eval set:")
print(val_df.isnull().sum())
print("\nMissing values in test set:")
print(test_df.isnull().sum())

Training set: 91942 examples
Validation set: 19702 examples
Test set: 19702 examples

Class distribution:
Training set:
label
1    51.31%
0    48.69%
Name: proportion, dtype: object

Validation set:
label
1    51.31%
0    48.69%
Name: proportion, dtype: object

Test set:
label
1    51.31%
0    48.69%
Name: proportion, dtype: object

Sample HAM emails:


,subject,body,label
82912,Re: getting rid of mkproto.sh from Samba3,-----BEGIN PGP SIGNED MESSAGE-----\nHash: SHA1...,0
48117,Re: getting rid of mkproto.sh from Samba3,"On Sun, 2007-06-03 at 20:29 -0500, Gerald (Jer...",0
43559,re : original karamojong / jie language,leo connolly cross-posted a question about kar...,0



Sample SPAM emails:


,subject,body,label
89728,digital cameras - - toshiba / sony / hewlett p...,t\no d a y ' s\ns p e c\ni a l\nvisit : http :...,1
75960,As casselberry before concan,\n\nTHE ALERT IS ON!!!\n\n\nPromoting sym: CHV...,1
18122,Fun fun fun,\nDear 7c393e5e81dd091bc5c33e91a55e1458\n\nSum...,1



Missing values in train set:
sender      22722
receiver    23431
date        22739
subject       532
body            0
label           0
urls        22722
source          0
dtype: int64

Missing values in eval set:
sender      4920
receiver    5055
date        4925
subject      109
body           1
label          0
urls        4920
source         0
dtype: int64

Missing values in test set:
sender      4984
receiver    5137
date        4985
subject      112
body           0
label          0
urls        4984
source         0
dtype: int64


In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict
from unsloth.chat_templates import (
    get_chat_template
)

# Load the datasets
train_df = pd.read_csv(
    "/home/kosmas/projects/llm-in-cybersecurity/final-project/datasets/train_emails.csv"
)
val_df = pd.read_csv(
    "/home/kosmas/projects/llm-in-cybersecurity/final-project/datasets/val_emails.csv"
)
test_df = pd.read_csv(
    "/home/kosmas/projects/llm-in-cybersecurity/final-project/datasets/test_emails.csv"
)

# Filter columns and handle missing values
train_df = train_df[["body", "label"]].dropna(subset=["body"])
val_df = val_df[["body", "label"]].dropna(subset=["body"])
test_df = test_df[["body", "label"]].dropna(subset=["body"])

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
datasets = DatasetDict(
    {"train": train_dataset, "validation": val_dataset, "test": test_dataset}
)

# Finetuning

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForImageTextToText, BitsAndBytesConfig
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

model = FastLanguageModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

==((====))==  Unsloth 2025.4.8: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce GTX 1080. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.1. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Unsloth: Making `base_model.model.vision_tower.vision_model` require gradients


In [15]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [21]:
from unsloth.chat_templates import standardize_data_formats
dataset_train = standardize_data_formats(datasets["train"])
dataset_eval = standardize_data_formats(datasets["validation"])
dataset_test = standardize_data_formats(datasets["test"])

In [22]:
dataset_train[100]

{'body': "RootsSearch.net:  Online          Auction         Check Mail | Gen Directory | My Services | Query         Forums | Downloads | Book Store | Resources | Articles         Categories         All         New         Closing         Going!         All Closed         Post         Register         My Auction Tired of paying charges to Ebay, Yahoo          etc?         Now you don't have to!\xa0 Post your          genealogy auction FREE!\xa0 We're just like the big          boys! Auction                      Categories                     Accessories\xa0                      Books                      Paper Items                      Records                      Everything Else  Sell your item with buy it now, proxy bidding,            reserve and more! Sign Up! or           Visit Us! [                      FAQ ] [                      Suggest A Category ] [                      Change Registration ] [                      Lookup Username/Password ] [                      User Feedb

In [ ]:
def formatting_prompts_func(examples):
    # Check if we're processing a single example or a batch
    if isinstance(examples['body'], str):
        # Single example
        conversation = [
            {
                "role": "user",
                "content": f"Please classify the following email as `PHISHING` or `SAFE`:\n\n```{examples['body']}```",
            },
            {
                "role": "assistant",
                "content": "PHISHING" if examples['label'] == 1 else "SAFE",
            }
        ]
        text = tokenizer.apply_chat_template(
            conversation, tokenize=False, add_generation_prompt=False
        ).removeprefix("<bos>")
        return {"text": [text]}
    else:
        # Batch of examples
        texts = []
        for i in range(len(examples['body'])):
            conversation = [
                {
                    "role": "user",
                    "content": f"Please classify the following email as `PHISHING` or `SAFE`:\n\n```{examples['body'][i]}```",
                },
                {
                    "role": "assistant",
                    "content": "PHISHING" if examples['label'][i] == 1 else "SAFE",
                }
            ]
            text = tokenizer.apply_chat_template(
                conversation, tokenize=False, add_generation_prompt=False
            ).removeprefix("<bos>")
            texts.append(text)
        return {"text": texts}


formatting_prompts_func(dataset_train[0])

{'text': ["<start_of_turn>user\nPlease classify the following email as `PHISHING` or `SAFE`:\n\n```christmass s @ | e - w ! ndows xp home\nwe have everything !\nwindows x ' p professional 20 o 2 . . . . . . . . . . . 5 o\nadobe photoshop 7 . 0 . . . . . . . . . . . . . . . . . . . . . . . . 60\nmicrosoft office x ' p pro 2 oo 2 . . . . . . . . . . . . . . 60\ncorel draw graphics suite 11 . . . . . . . . . . . . . 60\nfull information : http : / / inequality . bestsoftshop . info /\nyour paypal account\narnold callahan\nstockbroker\nbiosupplynet , a sciquest company , rtp , 27709 , united states of america\nphone : 218 - 821 - 3963\nmobile : 747 - 674 - 4118\nemail : rxqlwxca @ gee - wiz . com\nthis message is beng sent to confirm your account . please do not reply directly to this message\nthis shareware is a 16 minute usage freeware\nnotes :\nthe contents of this reply is for attention and should not be depend marginalia\nhide midway hastings\ntime : sat , 11 dec 2004 00 : 56 : 56 + 0

In [32]:
# Test it on a single example first
test_output = formatting_prompts_func(dataset_train[0])
print(test_output["text"][0][:100])  # Print first 100 chars to verify format

# Now apply to all datasets
formatted_datasets = {
    "train": dataset_train.map(formatting_prompts_func, batched=True, remove_columns=dataset_train.column_names),
    "validation": dataset_eval.map(formatting_prompts_func, batched=True, remove_columns=dataset_eval.column_names),
    "test": dataset_test.map(formatting_prompts_func, batched=True, remove_columns=dataset_test.column_names)
}

<start_of_turn>user
Please classify the following email as `PHISHING` or `SAFE`:

```christmass s @ 


Map:   0%|          | 0/91942 [00:00<?, ? examples/s]

Map:   0%|          | 0/19701 [00:00<?, ? examples/s]

Map:   0%|          | 0/19702 [00:00<?, ? examples/s]

In [33]:
from trl import SFTTrainer, SFTConfig

# Set up the trainer with explicit padding and truncation
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_datasets["train"],
    eval_dataset=formatted_datasets["validation"],
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/91942 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/19701 [00:00<?, ? examples/s]

In [35]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/91942 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/19701 [00:00<?, ? examples/s]

In [36]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><start_of_turn>user\nPlease classify the following email as `PHISHING` or `SAFE`:\n\n```RootsSearch.net:  Online          Auction         Check Mail | Gen Directory | My Services | Query         Forums | Downloads | Book Store | Resources | Articles         Categories         All         New         Closing         Going!         All Closed         Post         Register         My Auction Tired of paying charges to Ebay, Yahoo          etc?         Now you don't have to!\xa0 Post your          genealogy auction FREE!\xa0 We're just like the big          boys! Auction                      Categories                     Accessories\xa0                      Books                      Paper Items                      Records                      Everything Else  Sell your item with buy it now, proxy bidding,            reserve and more! Sign Up! or           Visit Us! [                      FAQ ] [                      Suggest A Category ] [                      Change Registration ]

In [37]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                          PHISHING<end_of_turn>\n'

In [38]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce GTX 1080. Max memory = 8.0 GB.
5.57 GB of memory reserved.


In [39]:
trainer_stats = trainer.train()
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 91,942 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 16,394,240/4,000,000,000 (0.41% trained)


GPUTooOldForTriton: Found NVIDIA GeForce GTX 1080 which is too old to be supported by the triton GPU compiler, which is used as the backend. Triton only supports devices of CUDA Capability >= 7.0, but your device is of CUDA capability 6.1

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
# Display training statistics
print(f"Training completed in {trainer_stats.metrics['train_runtime']:.2f} seconds")
print(f"Training loss: {trainer_stats.metrics['train_loss']:.4f}")

# Memory usage statistics
if torch.cuda.is_available():
    gpu_stats = torch.cuda.get_device_properties(0)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    
    print(f"\nGPU: {gpu_stats.name}")
    print(f"Peak memory usage: {used_memory} GB / {max_memory} GB ({used_percentage}%)")